In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

import numpy as np

In [2]:
USE_CUDA = True

In [3]:
root = '/notebooks/sinica/dataset/'
train_data = root+'facial.train'
dev_data = root+'facial.dev'
test_data = root+'facial.test'

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {"B-Func": 0, "I-Func": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

EMBEDDING_DIM = 20
HIDDEN_DIM = 10
tagset_size = len(tag_to_ix)

In [4]:
transitions = nn.Parameter(torch.randn(tagset_size, tagset_size))

# These two statements enforce the constraint that we never transfer
# to the start tag and we never transfer from the stop tag
transitions.data[tag_to_ix[START_TAG], :] = -10000
transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

transitions

Parameter containing:
tensor([[ 1.1223e+00, -9.3176e-01,  1.1441e-01, -1.4887e-01, -1.0000e+04],
        [-2.6964e-01, -1.5689e+00,  1.0331e-01, -2.3395e-01, -1.0000e+04],
        [ 5.8823e-02,  1.3466e+00,  1.2402e+00,  8.4746e-01, -1.0000e+04],
        [-1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04, -1.0000e+04],
        [-1.4701e+00,  1.7715e+00, -1.0673e+00,  4.6063e-01, -1.0000e+04]])

In [5]:
def get_word_and_label(_content, start_w, end_w):
    word_list = []
    tag_list = []
    for word_set in _content[start_w:end_w]:
        word_list.append(word_set[0])
        tag_list.append(word_set[2:])
    
    return tuple(word_list), tuple(tag_list)

def split_to_list(content):
    init = 0
    word_list = []
    tag_list = []

    for i, c in enumerate(content):
        if c=='':
            words, tags = get_word_and_label(content, init, i)
            init = i+1
            word_list.append(words)
            tag_list.append(tags)
            
    return word_list, tag_list
    
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

def word2index(word_list_train):
    word_to_ix = {}
    for sentence in word_list_train:
        for word in sentence:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
                
    return word_to_ix
    
# ===========================================
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()

# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score +         torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [6]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):       
        hidden = torch.randn(2, 1, self.hidden_dim // 2)        
        return (hidden.cuda(), hidden.cuda())if USE_CUDA else (hidden,hidden)
        
    def forward(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

In [7]:
def _forward_alg(feats):
    feats = feats.cpu()
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.full((1, tagset_size), -10000.)
    # START_TAG has all of the score.
    init_alphas[0][tag_to_ix[START_TAG]] = 0.

    # Wrap in a variable so that we will get automatic backprop
    forward_var = init_alphas

    # Iterate through the sentence
    for feat in feats:
        alphas_t = []  # The forward tensors at this timestep
        for next_tag in range(tagset_size):
            # broadcast the emission score: it is the same regardless of
            # the previous tag
            emit_score = feat[next_tag].view(
                1, -1).expand(1, tagset_size)
            # the ith entry of trans_score is the score of transitioning to
            # next_tag from i
            trans_score = transitions[next_tag].view(1, -1)
            # The ith entry of next_tag_var is the value for the
            # edge (i -> next_tag) before we do log-sum-exp
            next_tag_var = forward_var + trans_score + emit_score
            # The forward variable for this tag is log-sum-exp of all the
            # scores.
            alphas_t.append(log_sum_exp(next_tag_var).view(1))
        forward_var = torch.cat(alphas_t).view(1, -1)
    terminal_var = forward_var + transitions[tag_to_ix[STOP_TAG]]
    alpha = log_sum_exp(terminal_var)
    return alpha

In [8]:
def _score_sentence(feats, tags):
    feats = feats.cpu()
    # Gives the score of a provided tag sequence
    score = torch.zeros(1)
    tags = torch.cat([torch.tensor([tag_to_ix[START_TAG]], dtype=torch.long), tags])
    for i, feat in enumerate(feats):
        score = score + transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
    score = score + transitions[tag_to_ix[STOP_TAG], tags[-1]]
    return score

In [9]:
def _viterbi_decode(feats):
    feats = feats.cpu()
    backpointers = []

    # Initialize the viterbi variables in log space
    init_vvars = torch.full((1, tagset_size), -10000.)
    init_vvars[0][tag_to_ix[START_TAG]] = 0

    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = init_vvars
    for feat in feats:
        bptrs_t = []  # holds the backpointers for this step
        viterbivars_t = []  # holds the viterbi variables for this step

        for next_tag in range(tagset_size):
            # next_tag_var[i] holds the viterbi variable for tag i at the
            # previous step, plus the score of transitioning
            # from tag i to next_tag.
            # We don't include the emission scores here because the max
            # does not depend on them (we add them in below)
            next_tag_var = forward_var + transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            bptrs_t.append(best_tag_id)
            viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
        # Now add in the emission scores, and assign forward_var to the set
        # of viterbi variables we just computed
        forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
        backpointers.append(bptrs_t)

    # Transition to STOP_TAG
    terminal_var = forward_var + transitions[tag_to_ix[STOP_TAG]]
    best_tag_id = argmax(terminal_var)
    path_score = terminal_var[0][best_tag_id]

    # Follow the back pointers to decode the best path.
    best_path = [best_tag_id]
    for bptrs_t in reversed(backpointers):
        best_tag_id = bptrs_t[best_tag_id]
        best_path.append(best_tag_id)
    # Pop off the start tag (we dont want to return that to the caller)
    start = best_path.pop()
    assert start == tag_to_ix[START_TAG]  # Sanity check
    best_path.reverse()
    return path_score, best_path

In [10]:
def predict(sentence, model):

    lstm_feats = model(sentence)

    # Find the best path, given the features.
    score, tag_seq = _viterbi_decode(lstm_feats)
    
    return score, tag_seq

In [11]:
with open(train_data, "r", encoding="utf-8") as f:
    content = f.read().splitlines()

In [12]:
word_list_train, tag_list_train = split_to_list(content)

In [13]:
word_to_ix = word2index(word_list_train)

In [14]:
prepare_sequence(word_list_train[0], word_to_ix)

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
         12,  13,  14,  15,  16,   0,   1,   2,   3,  17,   0,  18,
         19,  20,  21,  22,  23,  24,  25,   4,  26])

In [15]:
model = BiLSTM(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM).cuda()
para = [i for i in model.parameters()]+[transitions]
optimizer = optim.Adam(para, lr=0.01, weight_decay=1e-4)

In [16]:
to_train_sent = word_list_train[:50]
to_train_tag = tag_list_train[:50]

In [17]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [18]:
# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(word_list_train[0], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in tag_list_train[0]], dtype=torch.long)
    print(predict(precheck_sent.cuda(), model))
    print(precheck_tags)

(tensor(53.5254), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1])
tensor([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  0,  1,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2])


In [19]:
start = time.time()
# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(10):  
    for sentence, tags in zip(to_train_sent,to_train_tag):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        lstm_feats = model(sentence_in.cuda()).cpu()
        forward_score = _forward_alg(lstm_feats)
        gold_score = _score_sentence(lstm_feats, targets)
        loss = forward_score - gold_score
        

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()
        
    
    print('Epoch: ', epoch+1, '| train loss: %.4f' % loss ,'| ',timeSince(start) )

Epoch:  1 | train loss: 2.5703 |  0m 5s
Epoch:  2 | train loss: 1.5025 |  0m 11s
Epoch:  3 | train loss: 1.7501 |  0m 17s
Epoch:  4 | train loss: 0.3015 |  0m 23s
Epoch:  5 | train loss: 0.0767 |  0m 29s
Epoch:  6 | train loss: 0.0327 |  0m 34s
Epoch:  7 | train loss: 0.0226 |  0m 40s
Epoch:  8 | train loss: 0.0115 |  0m 46s
Epoch:  9 | train loss: 0.0127 |  0m 52s
Epoch:  10 | train loss: 0.0070 |  0m 57s


In [20]:
# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(word_list_train[51], word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in tag_list_train[51]], dtype=torch.long)
    print(predict(precheck_sent.cuda(), model))
    print(precheck_tags)

(tensor(180.0339), [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
tensor([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  0,  1,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2])


In [21]:
transitions

Parameter containing:
tensor([[-4.4764e+00, -4.9303e-01,  4.2499e+00,  3.9597e+00, -9.9951e+03],
        [ 6.0550e+00, -7.0271e+00, -5.3599e+00, -5.5480e+00, -9.9951e+03],
        [-5.5373e+00, -2.4758e+00,  5.8117e+00,  3.8192e+00, -9.9951e+03],
        [-9.9951e+03, -9.9951e+03, -9.9951e+03, -9.9951e+03, -9.9951e+03],
        [-6.9630e+00, -3.5367e+00,  4.2539e+00,  3.0406e-12, -9.9951e+03]])